In [1]:
import pandas as pd
import numpy as np
import csv
import gensim
import spacy
from spacy import displacy
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import regexp_tokenize
from nltk import sent_tokenize
from nltk.stem.porter import PorterStemmer
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
import nltk
import en_core_web_sm
nlp = en_core_web_sm.load()
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
import matplotlib.pyplot as plt
from scipy import sparse

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jiezou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/jiezou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jiezou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#data=pd.read_csv('/Users/jiezou/Desktop/sjsu/256/project256/medical_conversation.txt')
f= open('/Users/jiezou/Desktop/sjsu/256/project256/medical_conversation.txt',"r")
#f= open('/Users/jiezou/Desktop/sjsu/256/project256/cardiology1-very-difficult.txt',"r")

In [3]:
data=f.read()
#df=pd.DataFrame(data)
#print(data)

In [4]:
#data.duplicated().sum()
#text1=str(data.iloc[0,0])

In [5]:
#print(text1)

In [6]:
def OderList(lists):
    lists = list(set(lists))
    ordered_tokens = set()
    result = []
    for word in lists:
        if word not in ordered_tokens:
            ordered_tokens.add(word)
            result.append(word)
    lists=" ".join(result)
    return result    

In [7]:
def cleaning(text):
    import nltk   
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub('[a-zA-Z]+[0-9]$', " ", text)
    news = news.replace("-", "")
    # 2. Convert to lower case.
    news =  news.lower()
    #print("Convert to lower case:",news)
    # 3. Tokenize.
    #news_words = word_tokenize(news)
    news_words = regexp_tokenize(news, "[\w']+")
    #print("Tokenize:",news_words)   
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("english"))    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    #print(" Remove stop words:",words)
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    #print("Lemmentize",wordnet_lem)
    # 7. Stemming
    #stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
    #print("Stemming",stems)
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    #sent=" ".join(stems)
    return wordnet_lem

In [8]:
# show all the words in coversation
#order all words
def order_tokens(words):
    ordered_tokens = set()
    result = []
    for word in words:
        if word not in result:
            ordered_tokens.add(word)
            result.append(word)
    return result

In [9]:
clean_word=cleaning(data)

In [10]:
words_list=order_tokens(clean_word)
print(words_list)

['transcript', 'hi', 'dr', 'helen', 'white', "today's", '16th', 'june', "i'm", 'referral', 'please', 'reference', 'hw', 'ca', 'peak', 'updated', 'date', 'private', 'confidential', 'mr', 'jason', 'spring', 'insult', 'orthopedic', 'surgeon', 'whiskey', 'hospital', 'thomas', 'jones', 'birth', '22nd', '1972', 'address', 'house', 'road', 'liverpool', 'l1', '3p', 'telephone', 'number', '7742193218', 'picante', 'damaged', 'r', 'e', 'pray', 'could', 'see', '58', 'year', 'old', 'gentleman', 'nice', 'young', 'tell', 'since', 'buying', 'tight', 'footwear', 'suffered', 'left', 'medial', 'foot', 'arch', 'pain', 'waiting', 'march', "he's", 'received', 'new', 'benefit', 'physiotherapy', 'find', 'get', 'worse', 'receives', 'exercise', 'regime', 'examination', 'baby', 'tendinosis', 'somewhat', '10', 'midpoint', 'middle', 'wonder', 'resolved', 'plantar', 'fasciitis', 'would', 'steroid', 'injection', 'grateful', 'expert', 'opinion', 'thank', 'much', 'seeing', 'say', 'like', 'snapchat']


In [11]:
print(clean_word)

['transcript', 'hi', 'dr', 'helen', 'white', "today's", '16th', 'june', "i'm", 'referral', 'please', 'reference', 'hw', 'ca', 'peak', 'updated', "today's", 'date', 'private', 'confidential', 'mr', 'jason', 'spring', 'insult', 'orthopedic', 'surgeon', 'whiskey', 'hospital', 'thomas', 'jones', 'date', 'birth', '22nd', 'june', '1972', 'address', 'house', 'road', 'liverpool', 'l1', '3p', 'telephone', 'number', '7742193218', 'transcript', 'hi', 'dr', 'helen', 'white', "today's", '16th', 'june', "i'm", 'referral', 'please', 'reference', 'hw', 'ca', 'picante', "today's", 'date', 'private', 'confidential', 'mr', 'jason', 'spring', 'insult', 'orthopedic', 'surgeon', 'whiskey', 'hospital', 'damaged', 'r', 'e', 'thomas', 'jones', 'date', 'birth', '22nd', 'june', '1972', 'address', 'house', 'road', 'liverpool', 'l1', '3p', 'telephone', 'number', '7742193218', 'transcript', 'pray', 'could', 'see', '58', 'year', 'old', 'gentleman', 'nice', 'young', 'gentleman', 'tell', 'since', 'buying', 'tight', 'f

In [12]:
clean_sent=" ".join(words_list)
print(clean_sent)

transcript hi dr helen white today's 16th june i'm referral please reference hw ca peak updated date private confidential mr jason spring insult orthopedic surgeon whiskey hospital thomas jones birth 22nd 1972 address house road liverpool l1 3p telephone number 7742193218 picante damaged r e pray could see 58 year old gentleman nice young tell since buying tight footwear suffered left medial foot arch pain waiting march he's received new benefit physiotherapy find get worse receives exercise regime examination baby tendinosis somewhat 10 midpoint middle wonder resolved plantar fasciitis would steroid injection grateful expert opinion thank much seeing say like snapchat


In [13]:
import re
#find the telephone number
def FindTelNum(strings):
    list1 = []
    list2 = []
    pattern = '[1-9]'
    r = re.compile(pattern)
    for i in strings:
        if r.match(i):
            list1.append(i)
        else:
            if len(list1) > len(list2):
                list2 = [i for i in list1]
            list1 = []
    if len(list1) > len(list2):
        return list1
    else:
        return list2

In [14]:
TelNum=FindTelNum(clean_sent)
TelNum="".join(TelNum)

In [15]:
print("TelNumber:",TelNum)

TelNumber: 7742193218


In [16]:
#labels = [x.label_ for x in doc.ents]
#print(labels)
#Counter(labels)

In [17]:
#find time words
def FindTime(strings): 
    Time_list =[]
    for word in strings.ents:
        labels = word.label_    
        if labels == "DATE":
        #print(word)
            Time_list.append(str(word))        
    return Time_list   

In [18]:
doc = nlp(clean_sent)
time_words=FindTime(doc)
print("Date:",time_words)

Date: ['today', 'june', '22nd 1972', '58 year old']


In [19]:
#find pations names
#Dictionary of all names
nltk.download('names')
from nltk.corpus import names
overall_name = ([name for name in names.words('male.txt')] + [name for name in names.words('female.txt')])

[nltk_data] Downloading package names to /Users/jiezou/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [20]:
print(overall_name)

['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot', 'Abbott', 'Abby', 'Abdel', 'Abdul', 'Abdulkarim', 'Abdullah', 'Abe', 'Abel', 'Abelard', 'Abner', 'Abraham', 'Abram', 'Ace', 'Adair', 'Adam', 'Adams', 'Addie', 'Adger', 'Aditya', 'Adlai', 'Adnan', 'Adolf', 'Adolfo', 'Adolph', 'Adolphe', 'Adolpho', 'Adolphus', 'Adrian', 'Adrick', 'Adrien', 'Agamemnon', 'Aguinaldo', 'Aguste', 'Agustin', 'Aharon', 'Ahmad', 'Ahmed', 'Ahmet', 'Ajai', 'Ajay', 'Al', 'Alaa', 'Alain', 'Alan', 'Alasdair', 'Alastair', 'Albatros', 'Albert', 'Alberto', 'Albrecht', 'Alden', 'Aldis', 'Aldo', 'Aldric', 'Aldrich', 'Aldus', 'Aldwin', 'Alec', 'Aleck', 'Alejandro', 'Aleks', 'Aleksandrs', 'Alessandro', 'Alex', 'Alexander', 'Alexei', 'Alexis', 'Alf', 'Alfie', 'Alfonse', 'Alfonso', 'Alfonzo', 'Alford', 'Alfred', 'Alfredo', 'Algernon', 'Ali', 'Alic', 'Alister', 'Alix', 'Allah', 'Allan', 'Allen', 'Alley', 'Allie', 'Allin', 'Allyn', 'Alonso', 'Alonzo', 'Aloysius', 'Alphonse', 'Alphonso', 'Alston', 'Alton', 'Alvin', 'Alwin', 'Amadeus'

In [21]:
def Lower(strings):
    strings =" ".join(strings)
    strings = strings.lower()
    strings = re.sub('[a-zA-Z]+[0-9]$', " ", strings)
    strings = regexp_tokenize(strings, "[\w']+")
    return  strings

In [22]:
overall_name=Lower(overall_name)

In [23]:
print(overall_name)

['aamir', 'aaron', 'abbey', 'abbie', 'abbot', 'abbott', 'abby', 'abdel', 'abdul', 'abdulkarim', 'abdullah', 'abe', 'abel', 'abelard', 'abner', 'abraham', 'abram', 'ace', 'adair', 'adam', 'adams', 'addie', 'adger', 'aditya', 'adlai', 'adnan', 'adolf', 'adolfo', 'adolph', 'adolphe', 'adolpho', 'adolphus', 'adrian', 'adrick', 'adrien', 'agamemnon', 'aguinaldo', 'aguste', 'agustin', 'aharon', 'ahmad', 'ahmed', 'ahmet', 'ajai', 'ajay', 'al', 'alaa', 'alain', 'alan', 'alasdair', 'alastair', 'albatros', 'albert', 'alberto', 'albrecht', 'alden', 'aldis', 'aldo', 'aldric', 'aldrich', 'aldus', 'aldwin', 'alec', 'aleck', 'alejandro', 'aleks', 'aleksandrs', 'alessandro', 'alex', 'alexander', 'alexei', 'alexis', 'alf', 'alfie', 'alfonse', 'alfonso', 'alfonzo', 'alford', 'alfred', 'alfredo', 'algernon', 'ali', 'alic', 'alister', 'alix', 'allah', 'allan', 'allen', 'alley', 'allie', 'allin', 'allyn', 'alonso', 'alonzo', 'aloysius', 'alphonse', 'alphonso', 'alston', 'alton', 'alvin', 'alwin', 'amadeus'

In [41]:
#Dictionary of doctor names in the allover_names
doctor_names={'helen','bonny'} 
# We can defined it from the HRM system of the hospital or input from the doctor every time

#Dictionary of time words in the allover_names
time_names = set(time_words) # We can defined it from finding time function
#time_names={'may','june','july'} 

#print(time_names)

#Dictionary of address words in the allover_names
address_names={'thomas','jerry'} # We can defined it from finding address function

#other stopwords for PatientName
stops = set(nltk.corpus.stopwords.words("english")) 
other_stops ={'say','see'}

#Dictionary of name stopwords for patient 
PatientName_Stopwords=set.union(doctor_names,time_names,address_names,stops,other_stops)

print(PatientName_Stopwords)

{'that', 'couldn', 'any', 'against', 'today', 'is', 'each', "aren't", 'isn', 'should', 'which', 'having', 'be', 'ma', 'wasn', 'both', 'aren', 'theirs', 'jerry', 'because', 'does', 'own', 'bonny', '58 year old', 'than', 'and', "you've", 'are', 'myself', 've', 'll', 'am', 'to', "mightn't", 'there', 'needn', "don't", 'we', "should've", 'has', 'an', 'nor', 'ourselves', 'once', 'she', "you're", 'further', "isn't", 'mustn', "couldn't", 'before', 'i', 'this', 'so', 'themselves', 'here', 'mightn', 'after', "needn't", 'see', 'until', 'me', 'few', 'doing', 'our', 'them', 'through', 'been', 'with', 'helen', 'these', 'but', 'its', "wasn't", 'they', 'him', 'his', 'where', 'about', 'off', 'during', "hadn't", "won't", 'when', 'was', 'under', 'he', 'by', 'it', "it's", 'yourselves', 'june', "she's", "didn't", 'shan', 'most', "shan't", 'while', 'or', 'y', 'over', 'do', 'weren', 'too', "that'll", 'who', 'why', '22nd 1972', 'some', 'didn', 'were', 'wouldn', 'no', 'm', 'such', 'hadn', 's', 'did', 'yourself

In [42]:
def FindPatientName(strings):   
    Name_List = [w for w in  strings if w in overall_name]
    Name_List = [w for w in  Name_List if not w in PatientName_Stopwords]
    Name_List = list(set(Name_List))
    ordered_tokens = set()
    result = []
    for word in Name_List:
        if word not in ordered_tokens:
            ordered_tokens.add(word)
            result.append(word)
    Name=" ".join(result)
    return Name    

In [43]:
PatientName=FindPatientName(clean_word)

In [44]:
print("Name:", PatientName)

Name: jason


In [45]:
#find disease symptom words
#disease dictionary
disease_data=pd.read_csv('/Users/jiezou/Desktop/sjsu/256/project256/disease/sym_3.csv')

In [46]:
print(disease_data.head(10))

   _id                     symptom
0    1        Upper abdominal pain
1    2        Lower abdominal pain
2    4               Alcohol abuse
3    5       Anxiety (Nervousness)
4    6            Arm ache or pain
5    7           Back ache or pain
6    8           Bleeding tendency
7    9              Blood in vomit
8   11  Pain or soreness of breast
9   13              Chest pressure


In [47]:
disease_data.symptom=disease_data.symptom.astype('str')

In [48]:
disease_data.dtypes

_id         int64
symptom    object
dtype: object

In [49]:
disease_list_all=" ".join(disease_data.symptom)

In [50]:
#print(disease_list_all)

In [51]:
disease_list=cleaning(disease_list_all)

In [52]:
print(disease_list)

['upper', 'abdominal', 'pain', 'lower', 'abdominal', 'pain', 'alcohol', 'abuse', 'anxiety', 'nervousness', 'arm', 'ache', 'pain', 'back', 'ache', 'pain', 'bleeding', 'tendency', 'blood', 'vomit', 'pain', 'soreness', 'breast', 'chest', 'pressure', 'chill', 'change', 'behavior', 'cough', 'depressed', 'dizziness', 'double', 'vision', 'diplopia', 'ear', 'pressure', 'pain', 'ear', 'eye', 'pain', 'irritation', 'facial', 'pain', 'fainting', 'fever', 'fever', 'returning', 'traveler', 'fever', 'unknown', 'origin', 'flank', 'pain', 'frequent', 'urination', 'frequency', 'delusion', 'hallucination', 'headache', 'hive', 'hypothermia', 'low', 'temperature', 'incontinence', 'leaking', 'urine', 'insomnia', 'trouble', 'sleeping', 'skin', 'itching', 'kidney', 'pain', 'flank', 'pain', 'leg', 'ache', 'pain', 'swelling', 'leg', 'lethargy', 'sluggishness', 'mouth', 'pain', 'muscle', 'pain', 'nasal', 'bleeding', 'neck', 'ache', 'pain', 'neck', 'swelling', 'numbness', 'obesity', 'overdose', 'heart', 'pulsatio

In [53]:
#disease_list=OderList(disease_list)

In [54]:
print(disease_list)
disease_list

['upper', 'abdominal', 'pain', 'lower', 'abdominal', 'pain', 'alcohol', 'abuse', 'anxiety', 'nervousness', 'arm', 'ache', 'pain', 'back', 'ache', 'pain', 'bleeding', 'tendency', 'blood', 'vomit', 'pain', 'soreness', 'breast', 'chest', 'pressure', 'chill', 'change', 'behavior', 'cough', 'depressed', 'dizziness', 'double', 'vision', 'diplopia', 'ear', 'pressure', 'pain', 'ear', 'eye', 'pain', 'irritation', 'facial', 'pain', 'fainting', 'fever', 'fever', 'returning', 'traveler', 'fever', 'unknown', 'origin', 'flank', 'pain', 'frequent', 'urination', 'frequency', 'delusion', 'hallucination', 'headache', 'hive', 'hypothermia', 'low', 'temperature', 'incontinence', 'leaking', 'urine', 'insomnia', 'trouble', 'sleeping', 'skin', 'itching', 'kidney', 'pain', 'flank', 'pain', 'leg', 'ache', 'pain', 'swelling', 'leg', 'lethargy', 'sluggishness', 'mouth', 'pain', 'muscle', 'pain', 'nasal', 'bleeding', 'neck', 'ache', 'pain', 'neck', 'swelling', 'numbness', 'obesity', 'overdose', 'heart', 'pulsatio

['upper',
 'abdominal',
 'pain',
 'lower',
 'abdominal',
 'pain',
 'alcohol',
 'abuse',
 'anxiety',
 'nervousness',
 'arm',
 'ache',
 'pain',
 'back',
 'ache',
 'pain',
 'bleeding',
 'tendency',
 'blood',
 'vomit',
 'pain',
 'soreness',
 'breast',
 'chest',
 'pressure',
 'chill',
 'change',
 'behavior',
 'cough',
 'depressed',
 'dizziness',
 'double',
 'vision',
 'diplopia',
 'ear',
 'pressure',
 'pain',
 'ear',
 'eye',
 'pain',
 'irritation',
 'facial',
 'pain',
 'fainting',
 'fever',
 'fever',
 'returning',
 'traveler',
 'fever',
 'unknown',
 'origin',
 'flank',
 'pain',
 'frequent',
 'urination',
 'frequency',
 'delusion',
 'hallucination',
 'headache',
 'hive',
 'hypothermia',
 'low',
 'temperature',
 'incontinence',
 'leaking',
 'urine',
 'insomnia',
 'trouble',
 'sleeping',
 'skin',
 'itching',
 'kidney',
 'pain',
 'flank',
 'pain',
 'leg',
 'ache',
 'pain',
 'swelling',
 'leg',
 'lethargy',
 'sluggishness',
 'mouth',
 'pain',
 'muscle',
 'pain',
 'nasal',
 'bleeding',
 'neck',
 

In [55]:
def FindDisease(strings):   
    Disease_List = [w for w in  strings if w in disease_list]
    Disease_List = [w for w in  Disease_List if not w in DiseaseList_Stopwords]
    Disease_List=OderList(Disease_List)
    Disease_List=" ".join(Disease_List)
    return Disease_List  

In [56]:
#other stopwords for DiseasesList
stops = set(nltk.corpus.stopwords.words("english")) 
other_stops2 ={'birth','middle','see','baby'}

#Dictionary of  stopwords for DiseaseList
DiseaseList_Stopwords=set.union(doctor_names,time_names,address_names,stops,other_stops2)

print(DiseaseList_Stopwords)

{'that', 'couldn', 'any', 'against', 'today', 'is', 'each', "aren't", 'isn', 'should', 'which', 'having', 'be', 'ma', 'wasn', 'both', 'aren', 'middle', 'theirs', 'jerry', 'because', 'does', 'own', 'bonny', '58 year old', 'than', 'and', "you've", 'are', 'myself', 've', 'll', 'am', 'to', "mightn't", 'there', 'needn', "don't", 'we', "should've", 'has', 'an', 'nor', 'ourselves', 'once', 'she', "you're", 'further', "isn't", 'mustn', "couldn't", 'before', 'i', 'this', 'so', 'themselves', 'here', 'mightn', 'after', "needn't", 'see', 'until', 'me', 'few', 'doing', 'our', 'them', 'through', 'been', 'with', 'helen', 'these', 'but', 'its', "wasn't", 'they', 'him', 'his', 'where', 'about', 'off', 'during', "hadn't", "won't", 'when', 'was', 'under', 'he', 'by', 'it', "it's", 'yourselves', 'june', "she's", "didn't", 'shan', 'most', "shan't", 'while', 'or', 'y', 'over', 'do', 'weren', 'too', "that'll", 'who', 'why', '22nd 1972', 'some', 'didn', 'were', 'wouldn', 'no', 'm', 'such', 'hadn', 's', 'did',

In [57]:
FindDisease(clean_word)

'foot pain'